In [1]:
import pandas as pd
import datetime as DT
import pyspark as ps
from pyspark.mllib.clustering import KMeans, KMeansModel
from numpy import array

In [2]:
from pyspark import SparkContext
sc=SparkContext("local" , "MyApp")

In [4]:
adclicksDF=pd.read_csv('~/Capstone/GameData/flamingo-data/ad-clicks.csv')
adclicksDF=adclicksDF.rename(columns= lambda x: x.strip()) #removing whitespaces from headers
adClicks=adclicksDF[['userId','userSessionId']]
adClicks['adCount'] = 1

/home/cloudera/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [26]:
adClicksAgg = adClicks.groupby(['userId','userSessionId']).size().reset_index()
adClicksAgg.columns = ['userId','userSessionId','totalAdClicks'] #rename the columns
adClicksAgg.head(n=5)

,userId,userSessionId,totalAdClicks
0,1,5835,6
1,1,7847,4
2,1,10041,2
3,1,12713,3
4,1,16232,9


In [8]:
buyclicksDF=pd.read_csv('~/Capstone/GameData/flamingo-data/buy-clicks.csv')
buyclicksDF=buyclicksDF.rename(columns= lambda x: x.strip()) #removing whitespaces from headers
buyClicks=buyclicksDF[['userId','userSessionId','team','price']]

In [9]:
userDF=pd.read_csv('~/Capstone/GameData/flamingo-data/users.csv')
userDF=userDF.rename(columns= lambda x: x.strip()) #removing whitespaces from headers
users=userDF[['userId','dob','country']]
now=pd.Timestamp(DT.datetime.now())
users['dob']=(now - (pd.to_datetime(users['dob']))).astype('<m8[Y]')
users.columns=['userId','age','country']

/home/cloudera/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
combinedUserBuys=buyClicks.merge(users, on='userId')
combinedUserBuys.head(5)

,userId,userSessionId,team,price,age,country
0,1300,5820,9,3.0,18.0,GR
1,1300,5820,9,3.0,18.0,GR
2,1300,9689,9,2.0,18.0,GR
3,1300,9689,9,3.0,18.0,GR
4,1300,12334,9,3.0,18.0,GR


In [31]:
userBuysAndAdClicks = combinedUserBuys.merge(adClicksAgg, on=['userId','userSessionId'])
userBuysAndAdClicks.columns=['userId','userSessionId','teamId','price','age','country','totalAdClicks']
UserBuysAndAdClicks.head(5)

,userId,userSessionId,team,price,age,country,totalAdClicks
0,1300,5820,9,3.0,18.0,GR,3
1,1300,5820,9,3.0,18.0,GR,3
2,1300,9689,9,2.0,18.0,GR,6
3,1300,9689,9,3.0,18.0,GR,6
4,1300,12334,9,3.0,18.0,GR,8


In [15]:
teamDF=pd.read_csv('~/Capstone/GameData/flamingo-data/team.csv')
teamDF=teamDF.rename(columns= lambda x: x.strip()) #removing whitespaces from headers

In [17]:
team=teamDF[['teamId','strength']]
team.head(2)

,teamId,strength
0,79,0.774474
1,92,0.171926


In [32]:
userBuysStrengthsAndAdClicks=userBuysAndAdClicks.merge(team, on='teamId')
userBuysStrengthsAndAdClicks.head(3)

,userId,userSessionId,teamId,price,age,country,totalAdClicks,strength
0,1300,5820,9,3.0,18.0,GR,3,0.952176
1,1300,5820,9,3.0,18.0,GR,3,0.952176
2,1300,9689,9,2.0,18.0,GR,6,0.952176


In [33]:
gameclicksDF=pd.read_csv('~/Capstone/GameData/flamingo-data/game-clicks.csv')
gameclicksDF=gameclicksDF.rename(columns= lambda x: x.strip()) #removing whitespaces from headers
gameclicks=gameclicksDF[['userId','userSessionId','isHit','teamLevel']]
gameclicks.head(3)

,userId,userSessionId,isHit,teamLevel
0,1038,5916,0,1
1,1099,5898,0,1
2,899,5757,0,1


In [38]:
gameclicksAgg = gameclicks.groupby(['userId','userSessionId','teamLevel']).sum().reset_index()
gameclicksAgg.head(n=5)


,userId,userSessionId,teamLevel,isHit
0,0,23473,1,28
1,0,24943,2,35
2,0,28377,3,32
3,0,32029,4,29
4,0,37439,5,19


In [39]:
userBuysStrengthsAdAndGameClicks = userBuysStrengthsAndAdClicks.merge(gameclicksAgg, on =['userId','userSessionId'])

In [40]:
userBuysStrengthsAdAndGameClicks.head(10)

,userId,userSessionId,teamId,price,age,country,totalAdClicks,strength,teamLevel,isHit
0,1300,5820,9,3.0,18.0,GR,3,0.952176,1,1
1,1300,5820,9,3.0,18.0,GR,3,0.952176,1,1
2,1300,9689,9,2.0,18.0,GR,6,0.952176,3,6
3,1300,9689,9,3.0,18.0,GR,6,0.952176,3,6
4,1300,12334,9,3.0,18.0,GR,8,0.952176,4,5
5,1300,12334,9,3.0,18.0,GR,8,0.952176,4,5
6,1300,12334,9,3.0,18.0,GR,8,0.952176,4,5
7,1300,15816,9,3.0,18.0,GR,5,0.952176,5,9
8,1300,15816,9,1.0,18.0,GR,5,0.952176,5,9
9,1300,15816,9,5.0,18.0,GR,5,0.952176,5,9


In [41]:
userSessionDF=pd.read_csv('~/Capstone/GameData/flamingo-data/user-session.csv')
userSessionDF=userSessionDF.rename(columns= lambda x: x.strip()) #removing whitespaces from headers
userSession=userSessionDF[['userSessionId','platformType']]
userSession.head(3)

,userSessionId,platformType
0,5675,iphone
1,5675,iphone
2,5795,android


In [42]:
userPlatformBuysStrengthsAdAndGameClicks=userBuysStrengthsAdAndGameClicks.merge(userSession,on='userSessionId')
userPlatformBuysStrengthsAdAndGameClicks.head()

,userId,userSessionId,teamId,price,age,country,totalAdClicks,strength,teamLevel,isHit,platformType
0,1300,5820,9,3.0,18.0,GR,3,0.952176,1,1,android
1,1300,5820,9,3.0,18.0,GR,3,0.952176,1,1,android
2,1300,5820,9,3.0,18.0,GR,3,0.952176,1,1,android
3,1300,5820,9,3.0,18.0,GR,3,0.952176,1,1,android
4,1300,9689,9,2.0,18.0,GR,6,0.952176,3,6,android
